In [3]:
!pip install "torch>=2.0.1"
!pip install "huggingface-hub==0.25.2" "transformers==4.41.0"
!pip install "sentence-transformers==2.2.2"
!pip install -U langchain-community pinecone-client pipeline
!pip install "langchain==0.1.16" "langchain-core==0.1.53" "langchain-pinecone==0.0.3" "pydantic<2"

  Using cached transformers-4.41.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.41.0-py3-none-any.whl (9.1 MB)
Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.1
    Uninstalling transformers-4.26.1:
      Successfully uninstalled transformers-4.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.2.1 requires transformers<4.27.0,>=3.4.0, but you have transformers 4.41.0 which is incompatible.
  Using cached lang

In [4]:
!pip install PyMuPdf

In [5]:
!pip install spacy==3.8.0
!pip install scispacy
!python -m spacy download en_core_web_sm
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_scibert-0.5.1.tar.gz
!pip install ipywidgets requests pandas tqdm

  Using cached spacy-3.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached thinc-8.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
Reason for being yanked: model compatibility problem
Using cached spacy-3.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.4 MB)
Using cached thinc-8.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (920 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.12
    Uninstalling thinc-8.1.12:
      Successfully uninstalled thinc-8.1.12
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.0 whic

In [6]:
!pip install pyngrok
!pip install fastapi
!pip install uvicorn
!pip install requests
!pip install nest-asyncio

In [7]:
import os
import re
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

2025-08-26 20:18:33.448668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756239513.475936     294 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756239513.483737     294 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
current_retriever = None

# os.environ["PINECONE_API_KEY"] = PINECONEKEY

# print("Pinecone Env Key set successfully")

In [9]:
import io
import fitz
import json
import os
from tqdm import tqdm
from PIL import Image
import pytesseract
from typing import List

skip_empty_pages = False
min_chars_for_page = 1  # page should have at least 1 char


def page_text_from_dict(page):
    d = page.get_text("dict")  # (blocks, lines, spans)
    blocks = d.get("blocks", [])
    text_blocks = []
    for b in blocks:
        if b.get("type") != 0:  # Skipping blocks other than text
            continue
        bbox = b.get("bbox", [0, 0, 0, 0])  # [x0,y0,x1,y1] -> (left, top, right, bottom)
        block_lines = []
        for line in b.get("lines", []):
            line_text = " ".join(span.get("text", "") for span in line.get("spans", []))
            if line_text:
                block_lines.append(line_text)
        if block_lines:
            text_blocks.append((bbox[1], bbox[0], "\n".join(block_lines)))  # sort by y, then x

    # sort top->down then left->right
    text_blocks.sort(key=lambda t: (t[0], t[1]))

    page_text = "\n\n".join(b[2] for b in text_blocks)

    # Removing Hyphens bw two lines "exam-\nple" -> "example"
    page_text = re.sub(r"(\w)-\n(\w)", r"\1\2", page_text)

    # Replacing excessive newlines with only 2
    page_text = re.sub(r"\n{3,}", "\n\n", page_text).strip()
    return page_text


def ocr_page_with_pytesseract(page, zoom=2):
    # render at higher resolution for better OCR
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, alpha=False)  # alpha False => no alpha channel
    mode = "RGB" if pix.n >= 3 else "L"
    try:
        img = Image.frombytes(mode, [pix.width, pix.height], pix.samples)
    except Exception:
        # fallback via bytes -> PIL open
        img = Image.open(io.BytesIO(pix.tobytes()))
    # run OCR
    ocr_result = pytesseract.image_to_string(img)
    return ocr_result


def preprocess_text_pdf(pdf_bytes):
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    pages_info = []
    empty_pages = []
    combined_parts = []

    for pno in tqdm(range(len(doc)), desc="Extracting text"):  # Progress bar
        page = doc[pno]

        try:
            text = page_text_from_dict(page) or ""
        except Exception:
            text = page.get_text("text") or ""

        text_stripped = text.rstrip()

        # If page lacks text, attempt OCR on the page image(s)
        if len(text_stripped) < min_chars_for_page:
            ocr_text = ""
            try:
                ocr_text = ocr_page_with_pytesseract(page, zoom=2) or ""
                ocr_text = ocr_text.strip()
            except Exception:
                # If OCR/rendering failed, keep empty and log in text snippet
                ocr_text = ""

            if len(ocr_text) >= min_chars_for_page:
                # OCR found text -> treat page as having text
                pages_info.append({"page": pno + 1, "text": ocr_text})
            else:
                # truly empty (no text layer and OCR failed or empty)
                empty_pages.append(pno + 1)
                pages_info.append({"page": pno + 1, "text": ""})
            continue

        pages_info.append({"page": pno + 1, "text": text_stripped})

    print("✅ Extraction complete.")
    print(f"Pages processed: {len(doc)}; Empty pages: {empty_pages}")

    return {"response": pages_info}


def preprocess_multiple_pdfs(files: List[tuple]):
    response_list = []
    for filename, pdf_bytes in files:
        # reuse existing function to preserve behavior/format
        single_result = preprocess_text_pdf(pdf_bytes)  # {"response": pages_info}
        pages_info = single_result.get("response", [])
        response_list.append({"filename": filename, "page_info": pages_info})
    return {"response": response_list}

In [10]:
# merged_extractor.py
"""
Merged extractor:
- Input: OCR/text-json (your format with 'response' -> [{'filename', 'page_info': [{'page','text'}]}])
- NER-first extraction using scispaCy/spaCy and pattern matcher
- Regex fallback, local-LLM fallback (optional)
- TOP_K_BY_FEATURE ranking and selection
- Output schema:
  { "file_name": "...", "name": "...", "sections": { "<HEADING>": [{"chunk_text": "...", "page": n, "source": "..."}, ... ] } }
- every chunk includes "source" now.
"""

import re
import json
import os
import traceback
from datetime import datetime
from typing import Any, Dict, List, Tuple, Optional

# try spaCy / scispaCy
try:
    import spacy
    from spacy.matcher import Matcher
    try:
        import scispacy  # presence check only
    except Exception:
        scispacy = None
except Exception:
    spacy = None
    Matcher = None
    scispacy = None

# local LLM context (optional)
_LOCAL_LLM_CTX: Dict[str, Any] = {"ready": False, "tokenizer": None, "model": None, "name": None}

# Config
_HEADINGS = [
    "INDICATIONS", "INDICATIONS AND USAGE",
    "DOSAGE AND ADMINISTRATION", "DOSAGE", "ADMINISTRATION",
    "USE IN SPECIFIC POPULATIONS", "PREGNANCY", "LACTATION", "PEDIATRIC USE", "GERIATRIC USE",
    "CONTRAINDICATIONS",
    "WARNINGS AND PRECAUTIONS", "WARNINGS",
    "ADVERSE REACTIONS", "ADVERSE",
    "DRUG INTERACTIONS", "DRUG INTERACTION", "INTERACTIONS",
    "OVERDOSAGE", "OVERDOSE",
    "DESCRIPTION", "PRODUCT DESCRIPTION", "DESCRIPTION (PRODUCT)",
    "CLINICAL PHARMACOLOGY", "PHARMACOLOGY", "CLINICAL PHARM",
    "NONCLINICAL TOXICOLOGY", "NONCLINICAL", "TOXICOLOGY",
    "CLINICAL STUDIES", "STUDIES", "CLINICAL TRIALS",
    "REFERENCES", "BIBLIOGRAPHY",
    "HOW SUPPLIED/STORAGE AND HANDLING", "HOW SUPPLIED", "STORAGE", "HANDLING",
    "PATIENT COUNSELING INFORMATION", "PATIENT COUNSELING", "PATIENT INFORMATION",
]

_ADDITIONAL_HEADINGS = [
    "INDICATIONS AND USAGE",
    "DOSAGE AND ADMINISTRATION",
    "CONTRAINDICATIONS",
    "WARNINGS AND PRECAUTIONS",
    "ADVERSE REACTIONS",
    "DRUG INTERACTIONS",
    "USE IN SPECIFIC POPULATIONS",
    "OVERDOSAGE",
    "DESCRIPTION",
    "CLINICAL PHARMACOLOGY",
    "NONCLINICAL TOXICOLOGY",
    "CLINICAL STUDIES",
    "REFERENCES",
    "HOW SUPPLIED/STORAGE AND HANDLING",
    "PATIENT COUNSELING INFORMATION",
    "OTHER",
]

_SENT_SPLIT_RE = re.compile(r'(?<=[\.!?])\s+')
_DOSAGE_REGEXES = [
    re.compile(r"\b\d+(?:[\.,]\d+)?\s?(?:mg|mcg|μg|g)\b(?:\s*(?:/|per)\s*kg)?(?:[^\n\.;]{0,120})", re.I),
    re.compile(r"recommended dosage[:\s\-]*([^\n]{0,500})", re.I),
]
_INDICATION_KW = re.compile(r"\b(indicat|used to treat|treatment of|indicated for|use for|usage|purpose|intended use)\b", re.I)
_ADVERSE_KW = re.compile(r"\b(adverse reaction|adverse event|side effect|side effects)\b", re.I)
_DATE_RE = re.compile(r"(Revised|Revision|Revision date|Updated|Effective)\s*[:\-]?\s*(\w+\s+\d{1,2},\s*\d{4}|\d{4}-\d{2}-\d{2})", re.I)
_DRUG_NAME_PATTERN = re.compile(r"^(?P<brand>[A-Z][A-Z0-9a-z\-\s]{1,120})\s*(?P<bits>[\d\w\.\s]*)$", re.M)

# User-provided TOP_K_BY_FEATURE controls
TOP_K_BY_FEATURE = {
    "dosage": 20,
    "indications": 5,
    "warnings": 10,
    "side_effects": 10,
    "contraindications": 10,
    "drug_interactions": 10,
    "overdosage": 10,
    "description": 10,
    "clinical_pharmacology": 10,
    "nonclinical_toxicology": 10,
    "clinical_studies": 10,
    "references": 2,
    "how_supplied": 2,
    "patient_counseling": 2,
}

# ------------------- helpers to accept your text-json input -------------------
def _pages_from_text_json(text_json: Any) -> Tuple[Optional[List[Dict[str, Any]]], Optional[str]]:
    """
    Convert OCR/text JSON (your format) into pages list:
      [{'page': 1, 'text': '...', 'text_lower': '...'}, ...]
    Accepts dict/list/JSON-string/bytes.
    """
    if text_json is None:
        return None, None

    parsed = None
    if isinstance(text_json, (bytes, bytearray)):
        try:
            parsed = json.loads(text_json.decode("utf-8", errors="ignore"))
        except Exception:
            try:
                parsed = json.loads(text_json)
            except Exception:
                parsed = None
    elif isinstance(text_json, str):
        try:
            parsed = json.loads(text_json)
        except Exception:
            parsed = None
    elif isinstance(text_json, dict) or isinstance(text_json, list):
        parsed = text_json
    else:
        return None, None

    if parsed is None:
        return None, None

    # Accept top-level dict with 'response' -> list of file entries
    resp = None
    if isinstance(parsed, dict) and "response" in parsed and isinstance(parsed["response"], list):
        resp = parsed["response"]
    elif isinstance(parsed, list):
        resp = parsed
    else:
        return None, None

    # pick first file entry with page_info if possible
    file_entry = None
    if len(resp) == 1:
        file_entry = resp[0]
    else:
        for e in resp:
            if isinstance(e, dict) and e.get("page_info"):
                file_entry = e
                break
        if file_entry is None:
            file_entry = resp[0]

    if not isinstance(file_entry, dict):
        return None, None

    filename = file_entry.get("filename") or file_entry.get("name") or ''
    page_info = file_entry.get("page_info") or []

    pages: List[Dict[str, Any]] = []
    for pi in page_info:
        if not isinstance(pi, dict):
            continue
        pnum = pi.get("page")
        txt = pi.get("text") or ""
        if pnum is None:
            pnum = len(pages) + 1
        pages.append({"page": int(pnum), "text": txt, "text_lower": (txt or "").lower()})
    pages = sorted(pages, key=lambda x: int(x.get("page", 0)))
    return pages, filename

# ------------------- page mapping helpers -------------------
def _concat_pages_to_full_text(pages: List[Dict[str,Any]]) -> Tuple[str, List[Tuple[int,int,int]]]:
    parts = []
    for p in pages:
        parts.append(p.get("text","") or "")
    full = "\n".join(parts)
    ranges = []
    pos = 0
    for p in pages:
        txt = p.get("text","") or ""
        length = len(txt)
        ranges.append((pos, pos + length, p.get("page")))
        pos += length + 1
    return full, ranges

def _build_page_char_index_map(pages: List[Dict[str,Any]]) -> List[Tuple[int,int,int]]:
    ranges = []
    pos = 0
    for p in pages:
        txt = p.get("text","") or ""
        length = len(txt)
        ranges.append((pos, pos + length, p.get("page")))
        pos += length + 1
    return ranges

def _find_page_by_char_index(char_idx: Optional[int], pages: List[Dict[str,Any]], page_map: Optional[List[Tuple[int,int,int]]] = None) -> Optional[int]:
    if char_idx is None:
        return None
    if page_map is None:
        page_map = _build_page_char_index_map(pages)
    for s,e,pn in page_map:
        if s <= char_idx <= e:
            return pn
    return None

def _find_page_for_match(matched_text: str, pages: List[Dict[str, Any]]) -> Optional[int]:
    if not matched_text:
        return None
    needle = re.sub(r"\s+", " ", matched_text.strip())[:500].lower()
    if not needle:
        return None
    for p in pages:
        if needle in p.get("text_lower", ""):
            return p["page"]
    short = needle[:80]
    for p in pages:
        if short in p.get("text_lower", ""):
            return p["page"]
    return None

# ------------------- heading detection (small, robust implementation) -------------------
def _find_headings_in_pages(pages: List[Dict[str,Any]]) -> List[Dict[str,Any]]:
    """
    Returns list of sections detected in the document:
    [{'section': heading, 'text': section_text, 'start_page': n, 'end_page': m}, ...]
    Uses _HEADINGS list as canonical anchors; falls back to simple heuristics.
    """
    if not pages:
        return []
    full_text, _ = _concat_pages_to_full_text(pages)
    sections: List[Dict[str,Any]] = []
    # build combined heading regex (word-boundary, case-insensitive)
    heading_patterns = [re.escape(h) for h in _HEADINGS if h and len(h) > 2]
    if heading_patterns:
        combined = r'(' + r'|'.join(heading_patterns) + r')'
        for m in re.finditer(combined, full_text, re.I):
            try:
                name = m.group(0).strip()
                start = m.end()
                # find next heading occurrence to bound the section
                next_m = re.search(combined, full_text[start:], re.I)
                if next_m:
                    end = start + next_m.start()
                else:
                    end = min(len(full_text), start + 8000)
                snippet = full_text[start:end].strip()
                if not snippet:
                    continue
                start_page = _find_page_for_match(snippet[:400], pages)
                end_page = _find_page_for_match(snippet[-400:], pages) or start_page
                sections.append({"section": name.upper(), "text": snippet, "start_page": start_page, "end_page": end_page})
            except Exception:
                continue
    # if no sections found, fallback: split pages into pseudo-sections per page
    if not sections:
        for p in pages:
            text = p.get("text","") or ""
            if text.strip():
                sections.append({"section": f"PAGE {p.get('page')}", "text": text, "start_page": p.get("page"), "end_page": p.get("page")})
    return sections

# ------------------- text chunking helpers -------------------
def _make_paragraph_from_text_and_index(text: str,
                                        match_start_idx: Optional[int] = None,
                                        match_text: Optional[str] = None,
                                        max_sentences: int = 6,
                                        max_chars: int = 1600) -> Optional[str]:
    if not isinstance(text, str) or not text.strip():
        return None
    idx = match_start_idx
    if idx is None and match_text:
        try:
            needle = re.sub(r"\s+", " ", match_text.strip())
            pos = text.lower().find(needle.lower())
            if pos >= 0:
                idx = pos
        except Exception:
            idx = None
    if idx is None:
        truncated = text.strip()[:max_chars]
        last_punc = max(truncated.rfind("."), truncated.rfind("!"), truncated.rfind("?"))
        if last_punc > int(len(truncated) * 0.6):
            return truncated[: last_punc + 1].strip()
        return truncated.strip()
    half = max_chars // 2
    start = max(0, idx - half)
    end = min(len(text), idx + half)
    window = text[start:end].strip()
    try:
        sents = _SENT_SPLIT_RE.split(window)
        if len(sents) > max_sentences:
            cum = 0
            target = idx - start
            sent_idx = 0
            for i, s in enumerate(sents):
                cum += len(s) + 1
                if cum >= target:
                    sent_idx = i
                    break
            left = max(0, sent_idx - (max_sentences // 2))
            right = min(len(sents), left + max_sentences)
            sel = " ".join(sents[left:right]).strip()
            if len(sel) > max_chars:
                return sel[:max_chars].strip()
            return sel
        else:
            combined = " ".join(sents).strip()
            if len(combined) > max_chars:
                return combined[:max_chars].strip()
            return combined
    except Exception:
        recommended = window[:max_chars].strip()
        return recommended

# ------------------- spaCy loader (prioritize scispaCy) -------------------
def _safe_load_spacy(model_name: str = "en_core_sci_scibert", prefer_scispacy: bool = True):
    if spacy is None:
        return None
    try:
        candidates = []
        scispacy_candidates = [
            "en_core_sci_scibert",
            "en_core_sci_lg",
            "en_core_sci_md",
            "en_core_sci_sm",
        ]
        fallback_spacy = ["en_core_web_sm"]
        if prefer_scispacy and scispacy is not None:
            candidates.extend(scispacy_candidates)
        if model_name and model_name not in candidates:
            candidates.insert(0, model_name)
        candidates.extend([c for c in fallback_spacy if c not in candidates])

        nlp = None
        for cand in candidates:
            try:
                nlp = spacy.load(cand)
                break
            except Exception:
                nlp = None
                continue

        if nlp is None:
            try:
                nlp = spacy.load("en_core_web_sm")
            except Exception:
                return None

        try:
            meta_name = (nlp.meta.get("name") or "").lower()
            is_scispacy_model = ("core_sci" in meta_name) or ("scibert" in meta_name)
        except Exception:
            is_scispacy_model = False

        # slim down non-scispacy models but keep ner
        if not is_scispacy_model:
            for comp in ("tagger", "parser"):
                if comp in nlp.pipe_names:
                    try:
                        nlp.remove_pipe(comp)
                    except Exception:
                        pass
        # ensure sentencizer
        try:
            if "sentencizer" not in nlp.pipe_names:
                try:
                    nlp.add_pipe("sentencizer", first=True)
                except Exception:
                    from spacy.pipeline import Sentencizer
                    nlp.add_pipe("sentencizer", first=True)
        except Exception:
            pass

        try:
            nlp.max_length = max(getattr(nlp, "max_length", 1000000), 10_000_000)
        except Exception:
            pass

        return nlp
    except Exception:
        return None

# ------------------- build matcher -------------------
def _build_feature_matcher(nlp):
    if nlp is None:
        return None
    m = Matcher(nlp.vocab)

    m.add("DOSAGE_MG", [[{"LIKE_NUM": True}, {"LOWER": {"IN": ["mg", "mcg", "μg", "g"]}}]])
    pop_patterns = [[{"LOWER": "pediatric"}], [{"LOWER": "children"}], [{"LOWER": "adult"}], [{"LOWER": "geriatric"}], [{"LOWER":"pregnant"}], [{"LOWER":"pregnancy"}], [{"LOWER":"breastfeeding"}]]
    m.add("POPULATION", pop_patterns)

    ind_patterns = [
        [{"LOWER":"indicated"}],
        [{"LOWER":"indication"}],
        [{"LOWER":"indications"}],
        [{"LOWER":"used"},{"LOWER":"to"},{"LOWER":"treat"}],
        [{"LOWER":"treatment"},{"LOWER":"of"}],
        [{"LOWER":"indicated"},{"LOWER":"for"}],
        [{"LOWER":"use"},{"LOWER":"for"}],
        [{"LOWER":"used"},{"LOWER":"for"}],
        [{"LOWER":"usage"}],
        [{"LOWER":"purpose"}],
        [{"LOWER":"intended"},{"LOWER":"use"}],
        [{"LOWER":"indications"},{"LOWER":"and"},{"LOWER":"usage"}],
    ]
    m.add("INDICATION", ind_patterns)

    warn_patterns = [
        [{"LOWER":"warning"}],
        [{"LOWER":"warnings"}],
        [{"LOWER":"precaution"}],
        [{"LOWER":"precautions"}],
        [{"LOWER":"black-box"}],
        [{"LOWER":"black"},{"LOWER":"box"}],
        [{"LOWER":"boxed"},{"LOWER":"warning"}],
        [{"LOWER":"try"},{"LOWER":"avoiding"}],
        [{"LOWER":"never"},{"LOWER":"use"}],
        [{"LOWER":"do"},{"LOWER":"not"},{"LOWER":"use"},{"LOWER":"if"}],
        [{"LOWER":"do"},{"LOWER":"not"},{"LOWER":"use"}],
        [{"LOWER":"allergy"}],
        [{"LOWER":"allergies"}],
    ]
    m.add("WARNING", warn_patterns)

    adverse_patterns = [
        [{"LOWER":"adverse"},{"LOWER":"reaction"}],
        [{"LOWER":"adverse"},{"LOWER":"event"}],
        [{"LOWER":"side"},{"LOWER":"effect"}],
        [{"LOWER":"side"},{"LOWER":"effects"}],
        [{"LOWER":"adverse"}],
    ]
    m.add("ADVERSE", adverse_patterns)

    contra_patterns = [
        [{"LOWER":"contra"}],
        [{"LOWER":"contraindication"}],
        [{"LOWER":"contraindicated"}],
        [{"LOWER":"contraindications"}],
        [{"LOWER":"who"},{"LOWER":"should"},{"LOWER":"not"},{"LOWER":"use"}],
        [{"LOWER":"do"},{"LOWER":"not"},{"LOWER":"recommend"}],
        [{"LOWER":"who"},{"LOWER":"should"},{"LOWER":"avoid"}],
        [{"LOWER":"not"},{"LOWER":"recommended"},{"LOWER":"for"}],
        [{"LOWER":"should"},{"LOWER":"not"},{"LOWER":"be"},{"LOWER":"used"}],
    ]
    m.add("CONTRAINDICATION", contra_patterns)

    dir_patterns = [
        [{"LOWER":"direction"}],
        [{"LOWER":"directions"}],
        [{"LOWER":"directions"},{"LOWER":"for"},{"LOWER":"use"}],
        [{"LOWER":"administration"}],
        [{"LOWER":"how"},{"LOWER":"to"},{"LOWER":"use"}]
    ]
    m.add("DIRECTION", dir_patterns)

    drugint_patterns = [
        [{"LOWER":"drug"},{"LOWER":"interaction"}],
        [{"LOWER":"drug"},{"LOWER":"interactions"}],
        [{"LOWER":"interact"},{"LOWER":"with"}],
        [{"LOWER":"concomitant"},{"LOWER":"medication"}],
        [{"LOWER":"concomitant"},{"LOWER":"medications"}],
        [{"LOWER":"coadministration"}],
    ]
    m.add("DRUG_INTERACTION", drugint_patterns)

    over_patterns = [
        [{"LOWER":"overdosage"}],
        [{"LOWER":"overdose"}],
        [{"LOWER":"poisoning"}],
        [{"LOWER":"exposure"},{"LOWER":"to"},{"LOWER":"an"}],
    ]
    m.add("OVERDOSAGE", over_patterns)

    desc_patterns = [
        [{"LOWER":"description"}],
        [{"LOWER":"description:"}],
        [{"LOWER":"chemical"},{"LOWER":"name"}],
        [{"LOWER":"composition"}],
        [{"LOWER":"formulation"}],
        [{"LOWER":"contains"}],
    ]
    m.add("DESCRIPTION", desc_patterns)

    ph_patterns = [
        [{"LOWER":"clinical"},{"LOWER":"pharmacology"}],
        [{"LOWER":"mechanism"},{"LOWER":"of"},{"LOWER":"action"}],
        [{"LOWER":"pharmacokinetics"}],
        [{"LOWER":"pharmacodynamics"}],
    ]
    m.add("CLINICAL_PHARM", ph_patterns)

    noncli_patterns = [
        [{"LOWER":"nonclinical"},{"LOWER":"toxicology"}],
        [{"LOWER":"carcinogenicity"}],
        [{"LOWER":"mutagenesis"}],
        [{"LOWER":"reproductive"},{"LOWER":"toxicity"}],
    ]
    m.add("NONCLINICAL", noncli_patterns)

    studies_patterns = [
        [{"LOWER":"clinical"},{"LOWER":"studies"}],
        [{"LOWER":"clinical"},{"LOWER":"trial"}],
        [{"LOWER":"study"},{"LOWER":"results"}],
    ]
    m.add("CLINICAL_STUDIES", studies_patterns)

    ref_patterns = [
        [{"LOWER":"references"}],
        [{"LOWER":"bibliography"}],
        [{"LOWER":"references:"}],
    ]
    m.add("REFERENCES", ref_patterns)

    supply_patterns = [
        [{"LOWER":"how"},{"LOWER":"supplied"}],
        [{"LOWER":"how-supplied"}],
        [{"LOWER":"storage"}],
        [{"LOWER":"storage"},{"LOWER":"and"},{"LOWER":"handling"}],
        [{"LOWER":"packaging"}],
    ]
    m.add("HOW_SUPPLIED", supply_patterns)

    counsel_patterns = [
        [{"LOWER":"patient"},{"LOWER":"counseling"}],
        [{"LOWER":"patient"},{"LOWER":"therapy"}],
        [{"LOWER":"counseling"},{"LOWER":"information"}],
    ]
    m.add("PATIENT_COUNSELING", counsel_patterns)

    return m

# ------------------- Regex search (long windows) -------------------
def _regex_search_feature(feature: str, full_text: str, pages: List[Dict[str,Any]], sections: List[Dict[str,Any]]) -> List[Dict[str,Any]]:
    out = []
    f = feature.lower()
    window_left = 600
    window_right = 900

    def _make_window_from_match(m):
        start = max(0, m.start() - window_left)
        end = min(len(full_text), m.end() + window_right)
        snippet = full_text[start:end]
        snippet = re.sub(r"\s+", " ", snippet).strip()
        page = _find_page_for_match(snippet, pages)
        para = _make_paragraph_from_text_and_index(full_text, match_start_idx=max(0, m.start()), max_sentences=8, max_chars=2000)
        return {"paragraph": para, "page": page, "match_text": snippet, "source": "regex"}

    if f == "dosage":
        for rx in _DOSAGE_REGEXES:
            for m in rx.finditer(full_text):
                out.append(_make_window_from_match(m))
    elif f in ("indications", "indication"):
        for m in _INDICATION_KW.finditer(full_text):
            start = max(0, m.start() - 400)
            end = min(len(full_text), m.end() + 900)
            snippet = full_text[start:end]
            page = _find_page_for_match(snippet, pages)
            para = _make_paragraph_from_text_and_index(full_text, match_start_idx=m.start(), max_sentences=8, max_chars=2000)
            out.append({"paragraph": para, "page": page, "match_text": snippet.strip(), "source": "regex"})
    elif f in ("side_effects","adverse","adverse_reactions"):
        for m in _ADVERSE_KW.finditer(full_text):
            start = max(0, m.start() - 400)
            end = min(len(full_text), m.end() + 900)
            snippet = full_text[start:end]
            page = _find_page_for_match(snippet, pages)
            para = _make_paragraph_from_text_and_index(full_text, match_start_idx=m.start(), max_sentences=8, max_chars=2000)
            out.append({"paragraph": para, "page": page, "match_text": snippet.strip(), "source": "regex"})
    elif f in ("contraindications","contra"):
        for m in re.finditer(r'(Contraindicat(?:ion|ions).{0,800})', full_text, re.I | re.S):
            snippet = m.group(0)
            page = _find_page_for_match(snippet, pages)
            para = _make_paragraph_from_text_and_index(full_text, match_start_idx=max(0, m.start()), max_sentences=8, max_chars=2000)
            out.append({"paragraph": para, "page": page, "match_text": snippet.strip(), "source": "regex"})
    elif f in ("warnings","warnings_and_precautions"):
        for m in re.finditer(r'(Warning[s]?|Precaution[s]?)(.{0,800})', full_text, re.I | re.S):
            snippet = m.group(0)
            page = _find_page_for_match(snippet, pages)
            para = _make_paragraph_from_text_and_index(full_text, match_start_idx=max(0, m.start()), max_sentences=8, max_chars=2000)
            out.append({"paragraph": para, "page": page, "match_text": snippet.strip(), "source": "regex"})
    elif f in ("drug_interactions","drug_interaction","interactions"):
        for m in re.finditer(r'(Drug Interact(?:ion|ions)|interact(?:s|ion)? with|concomitant (?:medication|medications)|coadminister)', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))
    elif f in ("overdosage","overdose"):
        for m in re.finditer(r'(Overdosage|Overdose|Poisoning|In case of overdose)(.{0,800})', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))
    elif f in ("clinical_pharmacology","pharmacology","clin_pharm"):
        for m in re.finditer(r'(Clinical Pharmacology|Mechanism of Action|Pharmacokinetics|Pharmacodynamics)(.{0,1000})', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))
    elif f in ("nonclinical_toxicology","nonclinical","toxicology"):
        for m in re.finditer(r'(Nonclinical Toxicology|Carcinogenicity|Mutagenesis|Fertility)(.{0,1000})', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))
    elif f in ("description","product_description"):
        for m in re.finditer(r'(Description|Description:|Formulation|Composition|Chemical name)(.{0,1000})', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))
    elif f in ("clinical_studies","studies"):
        for m in re.finditer(r'(Clinical Studies|Study Results|Clinical Trial)(.{0,1000})', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))
    elif f in ("references","bibliography"):
        for m in re.finditer(r'(References|Bibliography|References:)(.{0,800})', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))
    elif f in ("how_supplied","how_supplied_storage_and_handling","how supplied"):
        for m in re.finditer(r'(How Supplied|How-supplied|Storage and Handling|Storage|Packaging)(.{0,800})', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))
    elif f in ("patient_counseling","patient_information"):
        for m in re.finditer(r'(Patient Counseling|Patient Information|Counseling Information)(.{0,800})', full_text, re.I | re.S):
            out.append(_make_window_from_match(m))

    # dedupe by paragraph text (long windows)
    seen = set(); uniq=[]
    for it in out:
        key = ((it.get("paragraph") or it.get("match_text") or "")[:600]).strip().lower()
        if key and key not in seen:
            uniq.append(it); seen.add(key)
    return uniq

# ------------------- dosage parsing/validation -------------------
def _parse_and_validate_dosage_text(txt: str) -> Dict[str, Any]:
    if not txt or not isinstance(txt, str):
        return {"amount": None, "unit": None, "valid": False, "reason": "empty"}
    m = re.search(r"(\d+(?:[\.,]\d+)?)\s*(mg|mcg|μg|ug|g)\b", txt, re.I)
    if not m:
        return {"amount": None, "unit": None, "valid": False, "reason": "no numeric unit"}
    raw_amount = m.group(1).replace(",", ".")
    unit = m.group(2).lower().replace("ug", "mcg")
    try:
        amt = float(raw_amount)
    except Exception:
        return {"amount": None, "unit": unit, "valid": False, "reason": "bad number"}
    reason = None
    valid = True
    if unit == "mg" and amt > 10000:
        valid = False; reason = "implausible >10000 mg"
    if unit == "g" and amt > 1000:
        valid = False; reason = "implausible >1000 g"
    if unit in ("mcg", "μg") and amt > 1000000:
        valid = False; reason = "implausible micrograms"
    if re.search(r"/\s?kg|per\s?kg", txt, re.I):
        if unit in ("mg",) and amt > 200:
            valid = False; reason = "implausible >200 mg/kg"
    return {"amount": amt, "unit": unit, "valid": valid, "reason": reason}

# ------------------- NER extraction (scispaCy-first, long paragraphs) -------------------
def _ner_extract_features(full_text: str, pages: List[Dict[str, Any]], nlp, matcher) -> Dict[str, List[Dict[str, Any]]]:
    """
    NER-first extraction prioritized for scispaCy models.
    Returns dict of lists per feature with dicts: {"paragraph":..., "page":..., "match_text":..., "source":...}
    """
    out = {
        "indications": [], "dosage": [], "warnings": [], "contraindications": [], "side_effects": [],
        "special_populations": [],
        "drug_interactions": [], "overdosage": [], "description": [], "clinical_pharmacology": [],
        "nonclinical_toxicology": [], "clinical_studies": [], "references": [], "how_supplied": [], "patient_counseling": []
    }
    if nlp is None:
        return out

    doc = nlp(full_text)
    _, full_page_map = _concat_pages_to_full_text(pages)

    model_meta = ""
    try:
        model_meta = (nlp.meta.get("name") or "").lower()
    except Exception:
        model_meta = ""
    is_scispacy_model = ("core_sci" in model_meta) or ("scibert" in model_meta)

    seen_keys = set()

    # 1) doc.ents
    for ent in getattr(doc, "ents", []):
        ent_text = ent.text.strip()
        if not ent_text:
            continue
        start_char = getattr(ent, "start_char", None)
        para = _make_paragraph_from_text_and_index(full_text, match_start_idx=start_char, match_text=ent_text, max_sentences=8, max_chars=2000)
        page = _find_page_by_char_index(start_char, pages, page_map=full_page_map)
        sent_text = ent.sent.text.strip() if hasattr(ent, "sent") and ent.sent is not None else ent_text
        low = sent_text.lower()
        src_label = "scispacy_ent" if is_scispacy_model else "ner_ent"

        # Heuristic placements
        if re.search(r"\b\d+(?:[\.,]\d+)?\s?(mg|mcg|μg|g)\b", ent_text, re.I) or re.search(r"\b(dose|dosage|tablet|capsule)\b", ent_text, re.I):
            key = ("dosage_ent", ent_text.lower()[:350])
            if key not in seen_keys:
                out["dosage"].append({"paragraph": para, "page": page, "match_text": ent_text, "source": src_label})
                seen_keys.add(key)
            continue
        if any(k in low for k in ("indicat", "used to treat", "indicated for", "treatment of", "use for", "used for", "intended use")):
            key = ("ind_ent", ent_text.lower()[:300])
            if key not in seen_keys:
                out["indications"].append({"paragraph": para, "page": page, "match_text": ent_text, "source": src_label})
                seen_keys.add(key)
            continue
        if any(k in low for k in ("adverse", "side effect", "side effects", "adverse effects")) or any(k in ent_text.lower() for k in ("rash","nausea","vomit","diarrh","dizziness","headache")):
            key = ("adverse_ent", ent_text.lower()[:300])
            if key not in seen_keys:
                out["side_effects"].append({"paragraph": para, "page": page, "match_text": ent_text, "source": src_label})
                seen_keys.add(key)
            continue
        if any(k in low for k in ("contraindicat", "do not use", "who should avoid", "should not be used")):
            key = ("contra_ent", ent_text.lower()[:300])
            if key not in seen_keys:
                out["contraindications"].append({"paragraph": para, "page": page, "match_text": ent_text, "source": src_label})
                seen_keys.add(key)
            continue
        if any(k in low for k in ("interact", "interaction", "concomitant", "coadminister")):
            key = ("drugint_ent", ent_text.lower()[:300])
            if key not in seen_keys:
                out["drug_interactions"].append({"paragraph": para, "page": page, "match_text": ent_text, "source": src_label})
                seen_keys.add(key)
            continue
        if any(k in ent_text.lower() for k in ("acid", "salt", "hydrate", "sodium", "chloride", "methyl", "phenyl", "derivative")):
            key = ("desc_ent", ent_text.lower()[:300])
            if key not in seen_keys:
                out["description"].append({"paragraph": para, "page": page, "match_text": ent_text, "source": src_label})
                seen_keys.add(key)
            continue
        # fallback: description candidate
        key = ("other_ent", ent_text.lower()[:300])
        if key not in seen_keys:
            out["description"].append({"paragraph": para, "page": page, "match_text": ent_text, "source": src_label})
            seen_keys.add(key)

    # 2) matcher patterns
    if matcher is not None:
        matches = matcher(doc)
        for mid, start, end in matches:
            label = nlp.vocab.strings[mid]
            span = doc[start:end]
            span_text = span.text.strip()
            if not span_text:
                continue
            start_char = getattr(span, "start_char", None)
            para = _make_paragraph_from_text_and_index(full_text, match_start_idx=start_char, match_text=span_text, max_sentences=8, max_chars=2000)
            page = _find_page_by_char_index(start_char, pages, page_map=full_page_map)
            src_label = "scispacy" if is_scispacy_model else "ner"
            entry = {"paragraph": para, "page": page, "match_text": span_text, "source": src_label}

            if label == "INDICATION":
                key = ("indication", span_text.lower()[:160])
                if key not in seen_keys:
                    out["indications"].append(entry); seen_keys.add(key)
            elif label == "DOSAGE_MG":
                key = ("dosage", span_text.lower()[:350])
                if key not in seen_keys:
                    out["dosage"].append(entry); seen_keys.add(key)
            elif label == "POPULATION":
                pop = span_text.lower()
                out["special_populations"].append({"population": pop, "paragraph": para, "page": page, "source": src_label})
            elif label == "WARNING":
                key = ("warning", span_text.lower()[:160])
                if key not in seen_keys:
                    out["warnings"].append(entry); seen_keys.add(key)
            elif label == "ADVERSE":
                key = ("adverse", span_text.lower()[:160])
                if key not in seen_keys:
                    out["side_effects"].append(entry); seen_keys.add(key)
            elif label == "CONTRAINDICATION":
                key = ("contra", span_text.lower()[:320])
                if key not in seen_keys:
                    out["contraindications"].append(entry); seen_keys.add(key)
            elif label == "DRUG_INTERACTION":
                key = ("drug_interaction", span_text.lower()[:160])
                if key not in seen_keys:
                    out["drug_interactions"].append(entry); seen_keys.add(key)
            elif label == "OVERDOSAGE":
                key = ("overdosage", span_text.lower()[:160])
                if key not in seen_keys:
                    out["overdosage"].append(entry); seen_keys.add(key)
            elif label == "DESCRIPTION":
                key = ("description", span_text.lower()[:160])
                if key not in seen_keys:
                    out["description"].append(entry); seen_keys.add(key)
            elif label == "CLINICAL_PHARM":
                key = ("clinical_pharmacology", span_text.lower()[:320])
                if key not in seen_keys:
                    out["clinical_pharmacology"].append(entry); seen_keys.add(key)
            elif label == "NONCLINICAL":
                key = ("nonclinical_toxicology", span_text.lower()[:320])
                if key not in seen_keys:
                    out["nonclinical_toxicology"].append(entry); seen_keys.add(key)
            elif label == "CLINICAL_STUDIES":
                key = ("clinical_studies", span_text.lower()[:160])
                if key not in seen_keys:
                    out["clinical_studies"].append(entry); seen_keys.add(key)
            elif label == "REFERENCES":
                key = ("references", span_text.lower()[:160])
                if key not in seen_keys:
                    out["references"].append(entry); seen_keys.add(key)
            elif label == "HOW_SUPPLIED":
                key = ("how_supplied", span_text.lower()[:160])
                if key not in seen_keys:
                    out["how_supplied"].append(entry); seen_keys.add(key)
            elif label == "PATIENT_COUNSELING":
                key = ("patient_counseling", span_text.lower()[:160])
                if key not in seen_keys:
                    out["patient_counseling"].append(entry); seen_keys.add(key)

    # 3) sentence-level heuristics
    for sent in doc.sents:
        s_text = sent.text.strip()
        if not s_text:
            continue
        start_char = getattr(sent, "start_char", None)
        para = _make_paragraph_from_text_and_index(full_text, match_start_idx=start_char, match_text=s_text, max_sentences=8, max_chars=2000)
        page = _find_page_by_char_index(start_char, pages, page_map=full_page_map)
        low = s_text.lower()
        src_label = "scispacy" if is_scispacy_model else "ner_sent"

        if any(k in low for k in ("indicat", "used to treat", "indicated for", "treatment of", "use for", "used for", "usage", "purpose", "intended use")):
            key = ("indication_sent", low[:160])
            if key not in seen_keys:
                out["indications"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("contraindicat", "do not use", "who should avoid", "should not be used")):
            key = ("contra_sent", low[:340])
            if key not in seen_keys:
                out["contraindications"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("adverse", "side effect", "side effects", "adverse effects")):
            key = ("adverse_sent", low[:320])
            if key not in seen_keys:
                out["side_effects"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("warning",'allergies','allergens',"precaution","allergy alert","black box","boxed warning")):
            key = ("warn_sent", low[:360])
            if key not in seen_keys:
                out["warnings"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ('direc', "recommended dosage", "directions",'dose','dosage','directions for use','suggested dosage')):
            key = ("dosage_sent", low[:800])
            if key not in seen_keys:
                out["dosage"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)

        if any(k in low for k in ("drug interaction","interact", "interaction", "concomitant", "coadministration")):
            key = ("drugint_sent", low[:320])
            if key not in seen_keys:
                out["drug_interactions"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("overdose", "overdosage", "poisoning", "exposure to")):
            key = ("overdose_sent", low[:320])
            if key not in seen_keys:
                out["overdosage"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("clinical pharmacology","pharmacokinetic", "pharmacodynamics", "mechanism of action", "pharmacology")):
            key = ("pharm_sent", low[:320])
            if key not in seen_keys:
                out["clinical_pharmacology"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("nonclinical", "toxicology","nonclinical toxicology", "carcinogenicity", "mutagenesis", "fertility")):
            key = ("noncl_sent", low[:320])
            if key not in seen_keys:
                out["nonclinical_toxicology"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("clinical study", "study results", "clinical studies", "trial")):
            key = ("clinstudy_sent", low[:320])
            if key not in seen_keys:
                out["clinical_studies"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("references", "bibliography", "reference")):
            key = ("ref_sent", low[:320])
            if key not in seen_keys:
                out["references"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("how supplied", "how-supplied", "storage", "packaging", "handling")):
            key = ("supply_sent", low[:320])
            if key not in seen_keys:
                out["how_supplied"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)
        if any(k in low for k in ("patient counseling", "patient information", "counseling information")):
            key = ("pc_sent", low[:320])
            if key not in seen_keys:
                out["patient_counseling"].append({"paragraph": para, "page": page, "match_text": s_text, "source": src_label}); seen_keys.add(key)

    return out

# ------------------- collect unmatched paragraphs -> OTHER -------------------
def _collect_unmatched_paragraphs(pages: List[Dict[str,Any]], result: Dict[str,Any], min_para_chars: int = 40, max_other_chars: Optional[int] = 5000) -> Dict[str,Any]:
    seen = set()
    for k, v in result.items():
        if k.startswith("_") or k in ("file_id", "medicine", "creator", "version_date", "revision_date", "special_populations"):
            continue
        if isinstance(v, list):
            for it in v:
                cand = (it.get("paragraph") or it.get("match_text") or "") or ""
                cand_norm = re.sub(r"\s+", " ", cand).strip().lower()
                if cand_norm:
                    seen.add(cand_norm[:1000])
        elif isinstance(v, dict):
            for subk, subv in v.items():
                if isinstance(subv, dict):
                    cand = (subv.get("paragraph") or subv.get("match_text") or "") or ""
                    cand_norm = re.sub(r"\s+", " ", cand).strip().lower()
                    if cand_norm:
                        seen.add(cand_norm[:1000])

    other_paragraphs = []
    seen_local = set()
    for p in pages:
        page_num = p.get("page")
        text = p.get("text","") or ""
        paras = [pp.strip() for pp in re.split(r"\n\s*\n+", text) if pp and pp.strip()]
        if not paras:
            paras = [ln.strip() for ln in text.splitlines() if ln.strip()]
        for para in paras:
            para_clean = re.sub(r"\s+", " ", para).strip()
            if len(para_clean) < min_para_chars:
                continue
            key = para_clean.lower()[:1000]
            if not key:
                continue
            if key in seen:
                continue
            if key in seen_local:
                continue
            contained = False
            for s in seen:
                if len(s) > 40 and s in key:
                    contained = True; break
                if len(key) > 40 and key in s:
                    contained = True; break
            if contained:
                continue
            other_paragraphs.append({"paragraph": para_clean, "page": page_num, "source": "other"})
            seen_local.add(key)

    if not other_paragraphs:
        full_text = " ".join([p.get("text","") or "" for p in pages])
        sents = re.split(r'(?<=[\.\!\?])\s+', full_text)
        for s in sents:
            s_clean = re.sub(r"\s+", " ", s).strip()
            if len(s_clean) < min_para_chars:
                continue
            k = s_clean.lower()[:1000]
            if k in seen or k in seen_local:
                continue
            page = _find_page_for_match(s_clean, pages)
            other_paragraphs.append({"paragraph": s_clean, "page": page, "source": "other"})
            seen_local.add(k)

    blob = "\n\n".join([p["paragraph"] for p in other_paragraphs])
    if max_other_chars is not None and isinstance(max_other_chars, int) and max_other_chars > 0:
        if len(blob) > max_other_chars:
            blob = blob[:max_other_chars] + "\n\n...[truncated]"
    return {"blob": blob, "paragraphs": other_paragraphs}

# ------------------- ranking & filter helper -------------------
def _section_matches_feature(section_name: str, feature: str) -> bool:
    if not section_name:
        return False
    s = section_name.lower()
    f = feature.lower()
    if f in ("dosage",):
        return any(k in s for k in ("dosage", "administration"))
    if f in ("indications",):
        return any(k in s for k in ("indication", "indications", "usage", "use"))
    if f in ("warnings",):
        return any(k in s for k in ("warning", "precaution"))
    if f in ("side_effects",):
        return any(k in s for k in ("adverse", "side effect", "side-effect"))
    if f in ("contraindications",):
        return any(k in s for k in ("contra", "who should not", "not recommended"))
    if f in ("drug_interactions",):
        return any(k in s for k in ("interaction", "drug interaction", "concomitant"))
    if f in ("overdosage",):
        return any(k in s for k in ("overdosage", "overdose"))
    if f in ("clinical_pharmacology",):
        return any(k in s for k in ("pharmacology","pharmacokinetics","mechanism"))
    if f in ("nonclinical_toxicology",):
        return any(k in s for k in ("nonclinical","toxicology","carcinogenicity"))
    if f in ("clinical_studies",):
        return any(k in s for k in ("study","clinical study","trial"))
    if f in ("references",):
        return "reference" in s or "bibliography" in s
    if f in ("how_supplied",):
        return any(k in s for k in ("how supplied", "storage", "handling"))
    if f in ("patient_counseling",):
        return any(k in s for k in ("patient counseling", "patient information"))
    if f in ("description",):
        return any(k in s for k in ("description", "composition", "formulation"))
    return False

def _filter_and_rank(items: List[Dict[str,Any]], feature: str, sections: List[Dict[str,Any]], pages: List[Dict[str,Any]], top_k: int = 5) -> List[Dict[str,Any]]:
    if not items:
        return []

    source_weights = {
        "scispacy_ent": 9.0,
        "scispacy": 5.0,
        "ner_ent": 5.0,
        "ner": 4.0,
        "ner_sent": 2.5,
        "regex": 2.0,
        "llm": 1.0,
        "other": 0.5
    }

    scored = []
    section_texts = [(s.get("section","") or "", (s.get("text","") or "").lower(), s.get("start_page")) for s in (sections or [])]

    for it in items:
        para = (it.get("paragraph") or "") or ""
        mt = (it.get("match_text") or "") or ""
        page = it.get("page")
        src = (it.get("source") or "").lower()
        score = 0.0

        score += source_weights.get(src, 1.0)
        f = feature.lower()
        combined = (para + " " + mt).lower()
        if f.replace("_"," ") in combined:
            score += 2.0
        if f == "side_effects" and ("side effect" in combined or "adverse" in combined):
            score += 1.5
        if f == "drug_interactions" and ("interaction" in combined or "interact" in combined or "concomitant" in combined):
            score += 1.5

        try:
            if page:
                for sec_name, sec_text, sec_start in section_texts:
                    if sec_start is None:
                        continue
                    if abs(sec_start - page) <= 1 and _section_matches_feature(sec_name, feature):
                        score += 3.0
                        break
            for sec_name, sec_text, sec_start in section_texts:
                if not sec_text:
                    continue
                if para and para.strip().lower()[:300] in sec_text:
                    if _section_matches_feature(sec_name, feature):
                        score += 3.0
                        break
        except Exception:
            pass

        if feature == "dosage":
            parsed = it.get("_dosage_parsed") or {}
            if isinstance(parsed, dict) and parsed.get("valid"):
                score += 2.0

        length = len(para or mt or "")
        if length > 1000:
            score -= 0.5
        if length < 80:
            score += 0.2

        scored.append((score, page if page is not None else 9999, length, it))

    scored.sort(key=lambda x: (-x[0], x[1], x[2]))

    top_items = []
    seen = set()
    for sc, pg, ln, it in scored:
        key = ((it.get("paragraph") or "")[:500]).strip().lower()
        if not key:
            key = ((it.get("match_text") or "")[:500]).strip().lower()
        if key in seen:
            continue
        new_it = dict(it)
        new_it["_rank_score"] = round(float(sc), 3)
        top_items.append(new_it)
        seen.add(key)
        if len(top_items) >= max(1, int(top_k or 1)):
            break

    return top_items

# ------------------- Local LLM helpers (optional fallback) -------------------
def init_local_llm(model_name: str = "gpt2", load_8bit=True):
    if _LOCAL_LLM_CTX.get("ready") and _LOCAL_LLM_CTX.get("name") == model_name:
        return True
    try:
        from transformers import AutoTokenizer, AutoModelForCausalLM
        _LOCAL_LLM_CTX["tokenizer"] = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        _LOCAL_LLM_CTX.update({"ready": True, "model": model, "name": model_name})
        return True
    except Exception as exc:
        _LOCAL_LLM_CTX.update({"ready": False, "model": None, "tokenizer": None, "name": None, "error": str(exc)})
        return False

def _extract_json_from_text(text: str) -> Tuple[Optional[Dict[str, Any]], Optional[str]]:
    if not isinstance(text, str):
        return None, "model output not text"
    text = re.sub(r"^(?:json)?\s*", "", text.strip())
    text = re.sub(r"\s*$", "", text.strip())
    s = text.find("{")
    e = text.rfind("}")
    if s != -1 and e != -1 and e > s:
        candidate = text[s:e+1]
        try:
            parsed = json.loads(candidate)
            return parsed, None
        except Exception:
            try:
                cleaned = re.sub(r",\s*}\s*", "}", candidate)
                cleaned = re.sub(r",\s*\]\s*", "]", cleaned)
                parsed = json.loads(cleaned)
                return parsed, None
            except Exception as ee:
                return None, f"json parse error: {ee}; raw snippet: {candidate[:300]}"
    return None, "no json-like substring found"

def _local_llm_extract_single_feature(snippet_text: str, feature: str, max_new_tokens: int = 256, temperature: float = 0.0):
    if not _LOCAL_LLM_CTX.get("ready"):
        ok = init_local_llm()
        if not ok:
            return {"success": False, "error": "local LLM not available"}
    try:
        tokenizer = _LOCAL_LLM_CTX["tokenizer"]
        model = _LOCAL_LLM_CTX["model"]
        import torch
        prompt = (f"You are a strict extractor. Extract ONLY the '{feature}' field from the InputText below and RETURN only valid JSON.\n"
                  "If nothing relevant, return an empty list or null for that field.\n"
                  "The extracted value should be an array of objects with 'text' and optional 'page'.\n"
                  "InputText: '''\n" + snippet_text.strip()[:30000] + "\n'''\n"
                  f"Return JSON like: {{\"{feature}\":[{{\"text\":\"...\",\"page\":null}}]}}")
        enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=min(4096, tokenizer.model_max_length))
        device = next(model.parameters()).device
        enc = {k: v.to(device) for k, v in enc.items()}
        with torch.no_grad():
            out = model.generate(**enc, max_new_tokens=int(max_new_tokens), do_sample=False)
        txt = tokenizer.decode(out[0], skip_special_tokens=True)
        parsed, perr = _extract_json_from_text(txt)
        if parsed is not None:
            return {"success": True, "result": parsed}
        else:
            return {"success": False, "error": "could not parse JSON", "raw": txt[:2000], "parse_error": perr}
    except Exception as e:
        return {"success": False, "error": str(e)}

# ------------------- chunk collector for headings (fallback to sections) -------------------
def _collect_chunks_for_heading(heading: str, pages: List[Dict[str,Any]], sections: List[Dict[str,Any]], full_text: str, max_chars: int = 1200) -> List[Dict[str,Any]]:
    out: List[Dict[str,Any]] = []
    seen = set()
    for sec in sections:
        sec_name = (sec.get("section") or "").upper()
        if not sec_name:
            continue
        if heading.upper() in sec_name or any(tok in sec_name for tok in heading.upper().split()):
            paras = [p.strip() for p in re.split(r"\n\s*\n+", sec.get("text","")) if p.strip()]
            for p in paras:
                pshort = re.sub(r"\s+", " ", p)[:max_chars].strip()
                if not pshort:
                    continue
                page = sec.get("start_page") or sec.get("end_page") or _find_page_for_match(pshort, pages)
                key = pshort.lower()[:500]
                if key in seen:
                    continue
                out.append({"chunk_text": pshort, "page": page, "source": "collector"})
                seen.add(key)
                if len(out) >= 6:
                    break
    if not out:
        # fallback regex
        base = heading.split()[0] if heading else ""
        if base:
            pattern = re.compile(r'(' + re.escape(base) + r'.{0,800})', re.I | re.S)
            for m in pattern.finditer(full_text):
                snippet = m.group(0)
                snippet_clean = re.sub(r"\s+", " ", snippet).strip()
                snippet_short = snippet_clean[:max_chars]
                page = _find_page_for_match(snippet_short, pages)
                key = snippet_short.lower()[:500]
                if key in seen:
                    continue
                out.append({"chunk_text": snippet_short, "page": page, })
                seen.add(key)
                if len(out) >= 6:
                    break
    # deduped return items (no source)
    final = []
    seen2 = set()
    for it in out:
        k = (it.get("chunk_text") or "")[:400].strip().lower()
        if k and k not in seen2:
            final.append({"chunk_text": it.get("chunk_text"), "page": it.get("page")})
            seen2.add(k)
    return final
        
# ------------------- orchestrator: NER -> regex -> LLM, ranking, build schema -------------------
def process_text_json(text_json: Any, use_ner: bool = True, use_local_llm: bool = False, llm_budget_tokens: int = 256) -> Dict[str,Any]:
    """
    Main entry:
      - text_json: your OCR JSON (dict/list/string/bytes)
      - returns schema {"file_name":..., "name":..., "sections": {HEADING: [{"chunk_text","page","source"}, ...]}}
    """
    pages, filename = _pages_from_text_json(text_json)
    if pages is None:
        raise ValueError("Unrecognized text_json shape. Expecting {'response':[{'filename':..., 'page_info':[{'page':n,'text':...}]}]}")

    sections = _find_headings_in_pages(pages)
    # Build full_text aligned with spaCy char offsets
    full_text, full_page_map = _concat_pages_to_full_text(pages)

    # basic metadata (name = prominent uppercase line OR filename; no brand/generic)
    front = pages[0].get("text","") if pages else ""
    name = None
    for ln in (front or "").splitlines()[:10]:
        lnstr = ln.strip()
        if lnstr and len(lnstr) > 3 and lnstr.isupper():
            name = lnstr; break
    if not name:
        name = os.path.splitext(filename or "")[0] if filename else "UNKNOWN"

    # Prepare result dict
    result = {
        "file_name": filename or "",
        "name": name or "",
        # per-feature lists that we will populate (items keep 'source' field)
        "indications": [], "dosage": [], "warnings": [], "side_effects": [], "contraindications": [],
        "drug_interactions": [], "overdosage": [], "description": [], "clinical_pharmacology": [],
        "nonclinical_toxicology": [], "clinical_studies": [], "references": [], "how_supplied": [], "patient_counseling": [],
        "special_populations": {}
    }

    # 1) NER using scispaCy / spaCy matcher
    nlp = None; matcher = None; ner_out = {}
    if use_ner and spacy is not None:
        try:
            nlp = _safe_load_spacy("en_core_sci_scibert")
            if nlp:
                matcher = _build_feature_matcher(nlp)
                ner_out = _ner_extract_features(full_text, pages, nlp, matcher)
        except Exception:
            traceback.print_exc()
            ner_out = {}

    # helper to add ner entries to result (deduped) -- preserve source
    def _add_ner_to_result(key, items):
        if not items:
            return
        seen = { ((it.get("paragraph") or "")[:300]).strip() for it in result.get(key, []) }
        for it in items:
            para = it.get("paragraph") or it.get("match_text") or ''
            page = it.get("page")
            src = it.get("source") or "ner"
            txt = (para or "").strip()
            if txt and ((txt)[:300]).strip() not in seen:
                entry = {"paragraph": txt, "page": page, "source": src}
                result.setdefault(key, []).append(entry)
                seen.add(((txt)[:300]).strip())

    if ner_out:
        _add_ner_to_result("indications", ner_out.get("indications", []))
        _add_ner_to_result("dosage", ner_out.get("dosage", []))
        _add_ner_to_result("warnings", ner_out.get("warnings", []))
        _add_ner_to_result("side_effects", ner_out.get("side_effects", []))
        _add_ner_to_result("contraindications", ner_out.get("contraindications", []))
        _add_ner_to_result("drug_interactions", ner_out.get("drug_interactions", []))
        _add_ner_to_result("overdosage", ner_out.get("overdosage", []))
        _add_ner_to_result("description", ner_out.get("description", []))
        _add_ner_to_result("clinical_pharmacology", ner_out.get("clinical_pharmacology", []))
        _add_ner_to_result("nonclinical_toxicology", ner_out.get("nonclinical_toxicology", []))
        _add_ner_to_result("clinical_studies", ner_out.get("clinical_studies", []))
        _add_ner_to_result("references", ner_out.get("references", []))
        _add_ner_to_result("how_supplied", ner_out.get("how_supplied", []))
        _add_ner_to_result("patient_counseling", ner_out.get("patient_counseling", []))

        for sp in ner_out.get("special_populations", []):
            pop_key = sp.get("population")
            if pop_key and pop_key not in result["special_populations"]:
                result["special_populations"][pop_key] = {"paragraph": sp.get("paragraph"), "page": sp.get("page"), "source": sp.get("source", "ner")}
    else:
        # nothing from ner
        pass

    # 2) Regex fallback for missing features (preserve source "regex")
    features_to_check = ["indications","dosage","warnings","side_effects","contraindications",
                         "drug_interactions","overdosage","description","clinical_pharmacology",
                         "nonclinical_toxicology","clinical_studies","references","how_supplied","patient_counseling"]

    for feat in features_to_check:
        current = result.get(feat) or []
        if not current:
            try:
                candidates = _regex_search_feature(feat, full_text, pages, sections)
                if candidates:
                    seen = set()
                    for c in candidates:
                        keytxt = ((c.get("paragraph") or c.get("match_text") or "")[:400]).strip()
                        if keytxt and keytxt not in seen:
                            # store paragraph+page+source
                            result.setdefault(feat, []).append({"paragraph": c.get("paragraph"), "page": c.get("page"), "source": c.get("source", "regex")})
                            seen.add(keytxt)
            except Exception:
                traceback.print_exc()

    # 3) local LLM fallback for still-missing features (optional)
    missing_after_regex = [f for f in features_to_check if not result.get(f)]
    if missing_after_regex and use_local_llm:
        try:
            init_local_llm()
        except Exception:
            pass
        for feat in missing_after_regex:
            relevant_texts = []
            for sec in sections:
                if feat.upper() in sec.get("section","").upper() or any(k in sec.get("section","").upper() for k in ("DOSAGE","ADMINISTRATION","INDICATION","WARN","ADVERSE","CONTRA","INTERACT","OVERDOSE","PHARMACOLOGY","TOXICOLOGY","STUDIES","REFERENCE","HOW SUPPLIED","PATIENT")):
                    relevant_texts.append(sec.get("text",""))
            regex_cands = _regex_search_feature(feat, full_text, pages, sections)
            for rc in regex_cands[:3]:
                if rc.get("match_text"):
                    relevant_texts.append(rc["match_text"])
            snippet = "\n\n".join([t for t in relevant_texts if t])[:30000]
            if not snippet:
                snippet = full_text[:30000]
            try:
                llm_resp = _local_llm_extract_single_feature(snippet, feat, max_new_tokens=llm_budget_tokens)
            except Exception as e:
                llm_resp = {"success": False, "error": str(e)}
            if llm_resp.get("success"):
                parsed = llm_resp.get("result", {})
                arr = parsed.get(feat) or parsed.get("results", {}).get(feat) if isinstance(parsed, dict) else None
                if not arr:
                    arr = []
                normalized = []
                for v in arr:
                    if isinstance(v, str):
                        normalized.append({"paragraph": v, "page": None, "source": "llm"})
                    elif isinstance(v, dict):
                        txt = v.get("text") or v.get("dosage_text") or v.get("match_text") or v.get("paragraph")
                        if not txt:
                            continue
                        normalized.append({"paragraph": txt, "page": v.get("page"), "source": "llm"})
                if normalized:
                    seen = { ((it.get("paragraph") or "")[:300]).strip() for it in result.get(feat, []) }
                    for it in normalized:
                        k = ((it.get("paragraph") or "")[:300]).strip()
                        if k and k not in seen:
                            result.setdefault(feat, []).append(it)
                            seen.add(k)

    # post-process dedupe and dosage parsing (preserve source)
    for k in ["indications","dosage","warnings","side_effects","contraindications",
              "drug_interactions","overdosage","description","clinical_pharmacology",
              "nonclinical_toxicology","clinical_studies","references","how_supplied","patient_counseling"]:
        seen = set(); uniq = []
        for it in result.get(k, []):
            key = ((it.get("paragraph") or "")[:800]).strip()
            if key and key not in seen:
                if k == "dosage":
                    parsed = _parse_and_validate_dosage_text(it.get("paragraph") or "")
                    it["_dosage_parsed"] = parsed
                uniq.append(it); seen.add(key)
        result[k] = uniq

    # ------------------- apply TOP_K_BY_FEATURE ranking/filtering -------------------
    try:
        for k in TOP_K_BY_FEATURE:
            topk = TOP_K_BY_FEATURE.get(k, 1)
            filtered = _filter_and_rank(result.get(k, []), k, sections, pages, top_k=topk)
            # ensure dosage parsed
            if k == "dosage":
                for it in filtered:
                    if "_dosage_parsed" not in it:
                        it["_dosage_parsed"] = _parse_and_validate_dosage_text(it.get("paragraph") or "")
            # keep 'source' and keep paragraph+page only plus _meta
            cleaned = []
            for it in filtered:
                cleaned.append({
                    "paragraph": it.get("paragraph"),
                    "page": it.get("page"),
                    "source": it.get("source", "extracted"),
                    "_meta": {k: it.get("_dosage_parsed") if k=="dosage" else None, "_score": it.get("_rank_score")}
                })
            result[k] = cleaned
    except Exception:
        traceback.print_exc()

    # Build OTHER bucket
    try:
        other = _collect_unmatched_paragraphs(pages, result, min_para_chars=30, max_other_chars=8000)
        result["other"] = other
    except Exception:
        traceback.print_exc()
        result["other"] = {"blob": "", "paragraphs": []}
    FEATURE_TO_HEADING = {
        "indications": "INDICATIONS AND USAGE",
        "dosage": "DOSAGE AND ADMINISTRATION",
        "contraindications": "CONTRAINDICATIONS",
        "warnings": "WARNINGS AND PRECAUTIONS",
        "side_effects": "ADVERSE REACTIONS",
        "drug_interactions": "DRUG INTERACTIONS",
        "overdosage": "OVERDOSAGE",
        "description": "DESCRIPTION",
        "clinical_pharmacology": "CLINICAL PHARMACOLOGY",
        "nonclinical_toxicology": "NONCLINICAL TOXICOLOGY",
        "clinical_studies": "CLINICAL STUDIES",
        "references": "REFERENCES",
        "how_supplied": "HOW SUPPLIED/STORAGE AND HANDLING",
        "patient_counseling": "PATIENT COUNSELING INFORMATION",
    }

    sections_dict: Dict[str, List[Dict[str,Any]]] = {}

    # Prefer features (NER/regex/llm results) to populate headings
    for feat, heading in FEATURE_TO_HEADING.items():
        items = result.get(feat, []) or []
        if items:
            chunks = []
            for it in items:
                para = (it.get("paragraph") or "") or ""
                page = it.get("page")
                src = it.get("source", "extracted")
                # normalize paragraph whitespace and shorten to a chunk size (optional)
                chunk = re.sub(r"\s+", " ", para).strip()
                if not chunk:
                    continue
                chunks.append({"chunk_text": chunk[:2000], "page": page or 0})
            if chunks:
                sections_dict[heading] = chunks

    # For any canonical heading missing, attempt to extract using detected page sections or chunk collector
    for heading in _ADDITIONAL_HEADINGS:
        if heading in sections_dict:
            continue
        # attempt to collect chunks from detected page sections
        collected = _collect_chunks_for_heading(heading, pages, sections, full_text, max_chars=1200)
        if collected:
            sections_dict[heading] = [{"chunk_text": c.get("chunk_text"), "page": c.get("page")} for c in collected]


    # Add any other detected sections (not canonical)
    for sec in sections:
        sec_name = (sec.get("section") or "").upper()
        if not sec_name or sec_name in sections_dict:
            continue
        paras = [p.strip() for p in re.split(r"\n\s*\n+", sec.get("text","")) if p.strip()]
        paras_short = []
        for p in paras[:6]:
            pshort = re.sub(r"\s+", " ", p)[:1200].strip()
            if pshort:
                page = sec.get("start_page") or _find_page_for_match(pshort, pages)
                paras_short.append({"chunk_text": pshort, "page": page})
        if paras_short:
            sections_dict[sec_name] = paras_short

    # Finally, ensure OTHER is present and contains unmatched paragraphs (preserve source)
    other_pars = result.get("other", {}).get("paragraphs", [])
    if other_pars:
        sections_dict.setdefault("OTHER", [])
        for p in other_pars:
            sections_dict.setdefault("OTHER", [])
            sections_dict["OTHER"].append({"chunk_text": p.get("paragraph"), "page": p.get("page")})


    # Final object
    out = {"file_name": filename or "", "name": name or "", "sections": sections_dict}
    return out

def process_text_json_bulk(text_json: Any,
                           use_ner: bool = True,
                           use_local_llm: bool = False,
                           llm_budget_tokens: int = 256,
                           process_all: bool = False) -> Dict[str, Any]:
    """
    Processes either the first file (default) or all files in the input `text_json`.
    Returns: {"data": [<output for file1>, <output for file2>, ...]}
    """
    # normalize input -> parsed dict/list (reuse logic from _pages_from_text_json)
    parsed = None
    if isinstance(text_json, (bytes, bytearray)):
        try:
            parsed = json.loads(text_json.decode("utf-8", errors="ignore"))
        except Exception:
            parsed = json.loads(text_json)
    elif isinstance(text_json, str):
        try:
            parsed = json.loads(text_json)
        except Exception:
            parsed = None
    elif isinstance(text_json, dict) or isinstance(text_json, list):
        parsed = text_json
    if parsed is None:
        return {"data": []}

    # extract response list
    resp = None
    if isinstance(parsed, dict) and "response" in parsed and isinstance(parsed["response"], list):
        resp = parsed["response"]
    elif isinstance(parsed, list):
        resp = parsed
    else:
        return {"data": []}

    outputs = []
    # if only want first file (default) and at least one file present
    if not process_all:
        entry = resp[0] if resp else None
        if entry is None:
            return {"data": []}
        try:
            out = process_text_json({"response": [entry]}, use_ner=use_ner, use_local_llm=use_local_llm, llm_budget_tokens=llm_budget_tokens)
            return {"data": [out]}
        except Exception as e:
            return {"data": [{"file_name": entry.get("filename") or entry.get("name") or "", "error": str(e)}]}

    # process all entries
    for entry in resp:
        try:
            out = process_text_json({"response": [entry]}, use_ner=use_ner, use_local_llm=use_local_llm, llm_budget_tokens=llm_budget_tokens)
            outputs.append(out)
        except Exception as e:
            outputs.append({"file_name": entry.get("filename") or entry.get("name") or "", "error": str(e)})
    return {"data": outputs}


def _pages_to_input_json(pages: List[Dict[str,Any]], filename_hint: Optional[str]=None):
    page_info = [{"page": p.get("page"), "text": p.get("text")} for p in pages]
    return {"response": [{"filename": filename_hint or "", "page_info": page_info}]}
    
def extract_with_ner_then_regex_then_llm(sections, pages, filename_hint=None,
                                         use_ner=True, use_hf=False, use_local_llm=False,
                                         llm_budget_tokens: int = 256, process_all: bool = False):
    """
    Compatibility wrapper updated to optionally process multiple file entries.
    - If `pages` is provided (old behaviour) it will process that single pages-list.
    - Otherwise if `sections` is an OCR/json-like object (with 'response'), this will call process_text_json_bulk.
    - `process_all=True` will process all files in the response list.
    """
    # old behaviour: caller passed pages/list -> keep working
    if pages:
        input_json = _pages_to_input_json(pages, filename_hint)
        out = process_text_json(input_json, use_ner=use_ner, use_local_llm=use_local_llm, llm_budget_tokens=llm_budget_tokens)
        # map to old shape (single-file)
        heading_to_feature = {
            "INDICATIONS AND USAGE": "indications",
            "DOSAGE AND ADMINISTRATION": "dosage",
            "WARNINGS AND PRECAUTIONS": "warnings",
            "ADVERSE REACTIONS": "side_effects",
            "CONTRAINDICATIONS": "contraindications",
            "DRUG INTERACTIONS": "drug_interactions",
            "OVERDOSAGE": "overdosage",
            "DESCRIPTION": "description",
            "CLINICAL PHARMACOLOGY": "clinical_pharmacology",
            "NONCLINICAL TOXICOLOGY": "nonclinical_toxicology",
            "CLINICAL STUDIES": "clinical_studies",
            "REFERENCES": "references",
            "HOW SUPPLIED/STORAGE AND HANDLING": "how_supplied",
            "PATIENT COUNSELING INFORMATION": "patient_counseling",
        }
        feature_to_heading = {v: k for k, v in heading_to_feature.items()}
        result = {"file_id": out.get("file_name") or filename_hint or ""}
        result["medicine"] = {"brand": out.get("name") or "", "generic": ""}
        for feature in feature_to_heading:
            heading = feature_to_heading[feature]
            items = out.get("sections", {}).get(heading, [])
            mapped = [{"paragraph": it.get("chunk_text") or "", "page": it.get("page")} for it in items]
            result[feature] = mapped
        other = out.get("sections", {}).get("OTHER", [])
        result["other"] = {"paragraphs": [{"paragraph": it.get("chunk_text"), "page": it.get("page")} for it in other],
                           "blob": out.get("sections", {}).get("OTHER", [])}
        result["_diagnostics"] = {"compat_wrapper": True}
        return result

    # new behaviour: caller passed full OCR/json object in `sections` param (legacy name)
    return process_text_json_bulk(sections or {}, use_ner=use_ner, use_local_llm=use_local_llm,
                                  llm_budget_tokens=llm_budget_tokens, process_all=process_all)


# ------------------- small convenience wrapper if user wants to call process_text_json with JSON string/file -------------------
def process_text_json_from_file(path: str, use_ner: bool = True, use_local_llm: bool = False):
    with open(path, "r", encoding="utf-8") as f:
        j = json.load(f)
    return process_text_json(j, use_ner=use_ner, use_local_llm=use_local_llm)


example_ocr = {'response': [
    {'filename': 'AspirinImg.pdf', 'page_info': [{'page': 1, 'text': 'VALUMEDS ASPIRIN- aspirin tablet, delayed release\nSPIRIT PHARMACEUTICALS LLC\n\nASPIRIN 325 MG TABLETS\n\nActive ingredient (in each tablet)\nAspirin (NSAID)* 325 mg\n*nonsteroidal anti-inflammatory drug\n\nPurpose\nPain reliever\n\n¢ temporary relieves minor aches and pains due to:\ne headache\ne minor arthritis pain\n¢ toothache\ne menstrual pain\n\n© colds\n\n* or as recommended by a doctor\n\nReye’s syndrome: Children and teenagers who have or are recovering from chicken\npox or flu-like symptoms should not use this product. When using this product, if\nchanges in behavior with nausea and vomiting occur, consult a doctor because these\nsymptoms could be an early sign of Reye’s syndrome, a rare but serious illness.\n\nAllergy alert: Aspirin may cause a severe allergic reaction which may include:\ne hives\n\n¢ shock\n\ne facial swelling\n\ne asthma (wheezing)\n\nStomach bleeding warning: This product contains an NSAID, which may cause\nstomach bleeding.\n\nThe chance is higher if you\n\ne are age 60 or older\n\ne have had stomach ulcers or bleeding problems\n\ntake a blood thinning (anticoagulant) or steroid drug\n\ntake other drugs containing prescription or nonprescription NSAIDS(aspirin,\nibuprofen, naproxen or others)\n\ne have 3 or more alcoholic drinks every day while using this product\n\ne take more or for a longer time than directed.\n\nDo not use\n\nif you have ever had an allergic reaction to aspirin or any other pain reliever/fever'}, {'page': 2, 'text': 'reducer\n\nAsk a doctor before use if\nstomach bleeding warning applies to you\nyou have a history of stomach problems, such as heartburn\n\nyou have high blood pressure, heart disease, liver cirrhosis, or kidney disease\nyou are taking a diuretic\nyou have asthma\n\nAsk a doctor or phamracist before use if you are\ntaking a prescription drug for diabetis, gout or arthritis\n\nStop use and ask a doctor if\n\nallergic reaction occurs, Seek medical help right away.\nyou experience any of the following signs of stomach bleeding:\nfeel faint\n\nvomit blood\n\nhave bloody or black stools\n\nhave stomach pain that does not get better\n\npain gets worse or lasts more than 10 days\n\nredness or swelling is present\n\nany new symptom appear\n\nringing in the ears or a loss of hearing occurs\n\nThese could be signs of a serious condition.\n\nIf pregnant or breast-feeding,\n\nask a health professional before use. It is especially important not to use aspirin during\nthe last 3 months of pregnancy unless definitely directed to do so by a doctor because it\nmay cause problems in the unborn child or complications during delivery.\n\nKeep out of reach of children\n\nIn case of overdose, get medical help or contact a Poison Control! Center right away.\n1(800)222-1222\n\nDirections\n\ne adults and children 12 years of age and over: take 1 to 2 tablets every 4 hours, while\nsymptoms persist. Drink a full glass of water with each dose.\n\ne do not take more than 12 tablets in 24 hours unless directed by a doctor\n\ne children under 12 years of age: ask a doctor'}, {'page': 3, 'text': 'Other information\n\nstore below 25 °C (77 OF)\n\ne¢ Tampet Evident Feature: Do not use ff printed inner-seal beneath cap is missing or\nbroken\n\ncorn starch, croscarmellose sodium,D&C yellow #10 aluminum lake, FD&C yellow #6\naluminum lake, hypromellose, methacrylic acid copolymer, microcrystalline\ncellulose,mineral oil, polysorbate 80, simethicone, sodium hydroxide, sodium lauryl\nsulfate, talc,titanium dioxide, triethyl citrate\n\nQuestions or comments?\n1(888)333-9792\n\nPRINCIPAL DISPLAY PANEL\nVALUMEDS ™\nCompare to the active ingredient in ECOTRIN ® TABLETS*\n\nREGULAR STRENGTH\nASPIRIN\n\nPAIN RELIEVER (NSAID)\nSAFETY COATED\n\nPROTECT AGAINST\nSTOMACH UPSET\n100 TABLETS'}, {'page': 4, 'text': 'vi\n\n \n\n \n\n \n\n \n\n+ protects against stomach upset\n\nsafety coated\n\nnem qe aft 8 ou ew\nroad si BYDSHO BLL\nfCusn oye Aep Lave sy atounge eva 7) Seem ured\n\n        \n\n \n\n‘wow\nJa OROUBE\n\n   \n\nops xRuep\n‘pox Oes 9u} Jo suN0 DU) Yue ry eC INAS\nAq pqnUuBp 0 punpenuew yi S| prov Siu\n\nTvl\n\n    \n\n(NSAID)\n\nsO] e Aq peRowuKes $20\nopou _ uodio seu\na se\n‘oj sud put sag o BvOp AE\n$05)\n\nASPIR\nis\n\ncoated .\n* protects against stomach upset\n\n100 tablets (szsmgeacry\n\nsafety\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nItem Code (Source) NDC:682 10-2500\n\nHUMAN OTC DRUG\n\nORAL\n\n \n\nVALU MEDS ASPIRIN\naspirin tablet, delayed release\nProduct Information\n\nProduct Type\nRoute of Administration'}, {'page': 5, 'text': 'Active Ingredient/Active Moiety\n\nIngredient Name Basis of Strength\nASPIRIN (UNII: RIGCO5Y76E) (ASPIRIN - UNII:RL6CO5Y76E) ASPIRIN\n\nInactive Ingredients\nIngredient Name\nMICROCRYSTALLINE CELLULOSE (UNI; OP1R32D61U)\nMINERAL OIL (UNI: T5L8T28FGP)\nPOLYSORBATE 80 (UNII: 6(0ZP39ZG8H)\nDIMETHIC ONE (UNII: 92RU3N3Y10)\nSODIUM HYDROXIDE (UNII: 55X04QC32!)\nSODIUM LAURYL SULFATE (UNII: 368GB5141))\nTALC (UNII: 7SEV7J4R1U)\nTITANIUM DIOXIDE (UNII: 15FIX9V2)P)\nTRIETHYL CITRATE (UNII: 8296QXD6UM)\nMETHACRYLIC ACID - ETHYL ACRYLATE COPOLYMER (1:1) TYPE A (UNII: NX76LV5T8})\nSTARCH, CORN (UNII: 08232NY3S))\nCROSCARMELLOSE SODIUM (UNI: MZ28OL1HH48)\nD&C YELLOW NO. 10 (UNII: 35SW5USQ3G)\nFD&C YELLOW NO. 6 (UNII: H77VEI93A8)\nHYPROMELLOSES (UNII: 3NXW29V3WO)\n\nProduct Characteristics\n\nStrength\n325 mg\n\nStrength\n\nColor orange Score no score\nShape ROUND. Size 10mm\nFlavor Imprint Code 7\nContains\nPackaging\nAes Marketing Start Marketing End\n# item Code Package Description Date Date\nNDC:68210- i\n1 2500-1 1 in 1 CARTON 03/10/2020\n1 100 in 1 BOTTLE; Type 0: Not a Combination\nProduct\n\nMarketing Information\n\nMarketing Application Number or Monograph Marketing Start Marketing End\n\nCategory Citation Date Date\nOTC Monograph Drug M013 03/10/2020\n\nLabeler = SPIRIT PHARMACEUTICALS LLC (179621011)'}, {'page': 6, 'text': 'Revised: 12/2024 SPIRIT PHARMACEUTICALS LLC'}]}, 
    {'filename': 'AspirinTxt.pdf', 'page_info': [{'page': 1, 'text': 'VALUMEDS ASPIRIN- aspirin tablet, delayed release  \nSPIRIT PHARMACEUTICALS LLC\n\n----------\n\nASPIRIN 325 MG TABLETS\n\nActive ingredient (in each tablet)\n\nAspirin (NSAID)* 325 mg\n\n*nonsteroidal anti-inflammatory drug\n\nPurpose\n\nPain reliever\n\ntemporary relieves minor aches and pains due to: \nheadache \nminor arthritis pain\ntoothache\nmenstrual pain\ncolds\n or as recommended by a doctor\n\nReye’s syndrome:  Children and teenagers who have or are recovering from chicken\npox or flu-like symptoms should not use this product.  When using this product, if\nchanges in behavior with nausea and vomiting occur, consult a doctor because these\nsymptoms could be an early sign of Reye’s syndrome, a rare but serious illness. \n\nAllergy alert:   Aspirin may cause a severe allergic reaction which may include:\n\nhives\nshock\nfacial swelling\nasthma (wheezing)\n\nStomach bleeding warning:   This product contains an NSAID, which may cause \nstomach bleeding.\n\n  The chance is higher if you\n\nare age 60 or older\n have had stomach ulcers or bleeding problems\n take a blood thinning (anticoagulant) or steroid drug\n take other drugs containing prescription or nonprescription NSAIDS(aspirin,\nibuprofen, naproxen or others)\n have 3 or more alcoholic drinks every day while using this product\ntake more or for a longer time than directed.\n\nDo not use\n\nif you have ever had an allergic reaction to aspirin or any other pain reliever/fever'}, {'page': 2, 'text': 'reducer\n\nAsk a doctor before use if\n\nstomach bleeding warning applies to you\nyou have a history of stomach problems, such as heartburn\nyou have high blood pressure, heart disease, liver cirrhosis, or kidney disease\nyou are taking a diuretic\nyou have asthma\n\nAsk a doctor or phamracist before use if you are\n\ntaking a prescription drug for diabetis, gout or arthritis\n\nStop use and ask a doctor if\n\nallergic reaction occurs, Seek medical help right away.\nyou experience any of the following signs of stomach bleeding:\nfeel faint \nvomit blood\nhave bloody or black stools \nhave stomach pain that does not get better\npain gets worse or lasts more than 10 days\nredness or swelling is present\nany new symptom appear\nringing in the ears or a loss of hearing occurs\n\nThese could be signs of a serious condition.\n\nIf pregnant or breast-feeding,\n\nask a health professional before use. It is especially important not to use aspirin during\nthe last 3 months of pregnancy unless definitely directed to do so by a doctor because it\nmay cause problems in the unborn child or complications during delivery.\n\nKeep out of reach of children\n\nIn case of overdose, get medical help or contact a Poison Control Center right away.\n1(800)222-1222\n\nDirections\n\nadults and children 12 years of age and over: take 1 to 2 tablets every 4 hours, while\nsymptoms persist. Drink a full glass of water with each dose.\n do not take more than 12 tablets in 24 hours unless directed by a doctor\n children under 12 years of age: ask a doctor'}, {'page': 3, 'text': 'Other information\n\n0\n0\n\nstore below 25 C (77 F)\nTampet Evident Feature: Do not use if printed inner-seal beneath cap is missing or\nbroken\n\ncorn starch, croscarmellose sodium,D&C yellow #10 aluminum lake, FD&C yellow #6\naluminum lake, hypromellose, methacrylic acid copolymer, microcrystalline\ncellulose,mineral oil, polysorbate 80, simethicone, sodium hydroxide, sodium lauryl\nsulfate, talc,titanium dioxide, triethyl citrate\n\nQuestions or comments?\n\n1(888)333-9792\n\nPRINCIPAL DISPLAY PANEL\n\nTM\n\nVALUMEDS \n\nCompare to the active ingredient in ECOTRIN ® TABLETS*\n\nREGULAR STRENGTH\n\nASPIRIN\n\nPAIN RELIEVER (NSAID)\n\nSAFETY COATED \n\nPROTECT AGAINST\n\nSTOMACH UPSET\n\n100 TABLETS'}, {'page': 4, 'text': 'VALUMEDS ASPIRIN  \n\naspirin tablet, delayed release\n\nProduct Information\n\nProduct Type\nHUMAN OTC DRUG\nItem Code (Source)\nNDC:68210-2500\n\nRoute of Administration\nORAL'}, {'page': 5, 'text': 'Active Ingredient/Active Moiety\n\nIngredient Name\nBasis of Strength\nStrength\n\nASPIRIN  (UNII: R16CO5Y76E) (ASPIRIN - UNII:R16CO5Y76E)\nASPIRIN\n325 mg\n\nInactive Ingredients\n\nIngredient Name\nStrength\n\nMICROCRYSTALLINE CELLULOSE  (UNII: OP1R32D61U)\n \n\nMINERAL OIL  (UNII: T5L8T28FGP)\n \n\nPOLYSORBATE 80  (UNII: 6OZP39ZG8H)\n \n\nDIMETHICONE  (UNII: 92RU3N3Y1O)\n \n\nSODIUM HYDROXIDE  (UNII: 55X04QC32I)\n \n\nSODIUM LAURYL SULFATE  (UNII: 368GB5141J)\n \n\nTALC  (UNII: 7SEV7J4R1U)\n \n\nTITANIUM DIOXIDE  (UNII: 15FIX9V2JP)\n \n\nTRIETHYL CITRATE  (UNII: 8Z96QXD6UM)\n \n\nMETHACRYLIC ACID - ETHYL ACRYLATE COPOLYMER (1:1) TYPE A  (UNII: NX76LV5T8J)\n \n\nSTARCH, CORN  (UNII: O8232NY3SJ)\n \n\nCROSCARMELLOSE SODIUM  (UNII: M28OL1HH48)\n \n\nD&C YELLOW NO. 10  (UNII: 35SW5USQ3G)\n \n\nFD&C YELLOW NO. 6  (UNII: H77VEI93A8)\n \n\nHYPROMELLOSES  (UNII: 3NXW29V3WO)\n \n\nProduct Characteristics\n\nColor\norange\nScore\nno score\n\nShape\nROUND\nSize\n10mm\n\nFlavor\nImprint Code\nT\n\nContains\n    \n\nPackaging\n\n#\nItem Code\nPackage Description\nMarketing Start\n\nMarketing End\n\nDate\n\nDate\n\n1   NDC:68210-\n\n2500-1\n1 in 1 CARTON\n03/10/2020\n\n1\n100 in 1 BOTTLE; Type 0: Not a Combination\nProduct\n\nMarketing Information\n\nMarketing\n\nApplication Number or Monograph\n\nMarketing Start\n\nMarketing End\n\nCategory\n\nCitation\n\nDate\n\nDate\n\nOTC Monograph Drug\nM013\n03/10/2020\n\nLabeler -  SPIRIT PHARMACEUTICALS LLC (179621011)'}, {'page': 6, 'text': 'SPIRIT PHARMACEUTICALS LLC\n \nRevised: 12/2024'}]}]}

# import json


# out = process_text_json(example_ocr, use_ner=True, use_local_llm=False)

# out = process_text_json_bulk(example_ocr, process_all=True)   # returns {"data": [out1, out2, ...]}

# print(json.dumps(out, indent=2, ensure_ascii=False))


In [11]:
# DATA = {
#     "data":
#     [
#         {
#       "file_name": "AspirinImg.pdf",
#       "name": "VALUMEDS ASPIRIN 325 MG",
#       "sections": {
#         "INDICATIONS AND USAGE": [
#           {
#             "chunk_text": "Aspirin 325 mg tablets — analgesic for temporary relief of minor aches and pains such as headache, minor arthritis pain, toothache, and menstrual pain; also used for colds or as recommended by a doctor.",
#             "page": 1
#           },
#           {
#             "chunk_text": "Provides temporary relief of pain and is intended for short-term symptomatic relief when used as directed on the label or by a physician.",
#             "page": 1
#           }
#         ],
#         "DOSAGE AND ADMINISTRATION": [
#           {
#             "chunk_text": "Each tablet contains 325 mg of aspirin. Use according to label directions or as directed by a healthcare professional. Do not exceed recommended dose or duration.",
#             "page": 1
#           },
#           {
#             "chunk_text": "If symptoms persist or worsen, consult a doctor. Adults and adolescents should follow package dosing; consult physician for pediatric dosing.",
#             "page": 2
#           }
#         ],
#         "CONTRAINDICATIONS": [
#           {
#             "chunk_text": "Do not use if you have had an allergic reaction to aspirin or any other pain reliever/fever reducer (NSAID). History of severe hypersensitivity to aspirin is a contraindication.",
#             "page": 1
#           }
#         ],
#         "WARNINGS AND PRECAUTIONS": [
#           {
#             "chunk_text": "Reye’s syndrome warning: Children and teenagers who have or are recovering from chicken pox or flu-like symptoms should not use this product. Seek medical attention if changes in behavior with nausea and vomiting occur.",
#             "page": 1
#           },
#           {
#             "chunk_text": "Stomach bleeding warning: This product contains an NSAID and may cause stomach bleeding. Risk increases with age (over 60), history of ulcers, anticoagulant or steroid use, concurrent NSAIDs, heavy alcohol use, or taking more than directed.",
#             "page": 1
#           }
#         ],
#         "OTHER": [
#           {
#             "chunk_text": "Do not use if you have ever had an allergic reaction to aspirin or any other pain reliever/fever reducer.",
#             "page": 1
#           },
#           {
#             "chunk_text": "If stomach bleeding warning applies to you or you have history of heartburn or stomach problems, consult a doctor before use.",
#             "page": 2
#           }
#         ]
#     }
# },
#         {
#       "file_name": "IbuprofenImg.pdf",
#       "name": "VALUMEDS IBUPROFEN 200 MG",
#       "sections": {
#         "INDICATIONS AND USAGE": [
#           {
#             "chunk_text": "Ibuprofen 200 mg tablets — used for the relief of minor aches and pains due to headache, muscular aches, backache, menstrual cramps, minor arthritis pain, toothache, and the common cold.",
#             "page": 1
#           },
#           {
#             "chunk_text": "Also reduces fever and provides temporary relief when used according to the package directions or a doctor’s advice.",
#             "page": 1
#           }
#         ],
#         "DOSAGE AND ADMINISTRATION": [
#           {
#             "chunk_text": "Each tablet contains 200 mg of ibuprofen. Adults and children 12 years and over: take 1 tablet every 4 to 6 hours while symptoms persist. If pain or fever does not respond, 2 tablets may be used.",
#             "page": 1
#           },
#           {
#             "chunk_text": "Do not exceed 6 tablets in 24 hours unless directed by a doctor. Children under 12: consult a physician.",
#             "page": 2
#           }
#         ],
#         "CONTRAINDICATIONS": [
#           {
#             "chunk_text": "Do not use if you have had an allergic reaction to ibuprofen, aspirin, or other NSAIDs. Contraindicated in patients with a history of asthma, urticaria, or allergic-type reactions after taking NSAIDs.",
#             "page": 1
#           }
#         ],
#         "WARNINGS AND PRECAUTIONS": [
#           {
#             "chunk_text": "Stomach bleeding warning: This product contains an NSAID, which may cause severe stomach bleeding. Risk factors include age over 60, prior ulcers, concurrent anticoagulant use, or taking more than directed.",
#             "page": 1
#           },
#           {
#             "chunk_text": "Heart attack and stroke warning: NSAIDs, except aspirin, increase the risk of heart attack, heart failure, and stroke. Risk is higher with long use or in people with heart disease.",
#             "page": 2
#           }
#         ],
#         "OTHER": [
#           {
#             "chunk_text": "Do not use right before or after heart surgery.",
#             "page": 1
#           },
#           {
#             "chunk_text": "Ask a doctor before use if you have high blood pressure, heart disease, liver cirrhosis, kidney disease, or are taking diuretics.",
#             "page": 2
#           }
#         ]
#     }
# }
# ]
# }


In [12]:
# required packages:
# pip install langchain sentence-transformers pinecone-client

import os
import json
import hashlib
from datetime import datetime
from typing import List
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
pinecone_key = user_secrets.get_secret("R_PINECONE_KEY")

# Pinecone Configuration
PINECONE_API_KEY = pinecone_key

os.environ["PINECONE_API_KEY"] = pinecone_key

if not PINECONE_API_KEY:
    raise RuntimeError("Set PINECONE_API_KEY in environment")

# Embeddings (HuggingFace wrapper)
HF_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=HF_MODEL)

def push_into_db(DATA, user_id):
    dimension = 384
    def build_chunks_from_data(data):
        chunks = []
        file_name = data.get("file_name", "unknown.pdf")
        doc_name = data.get("name", "unknown")
        sections = data.get("sections", {})
        for section_name, items in sections.items():
            for item in items:
                chunk_text = item.get("chunk_text", "").strip()
                
                page = item.get("page")
                if not chunk_text:
                    continue
                metadata = {
                    "file_name": file_name.lower(),
                    "page": int(page) if page is not None else 0,
                    "name": doc_name.lower(),
                    "topic": section_name.lower()
                }
                record = {
                    "name": doc_name.lower(),
                    "topic": section_name.lower(),
                    "chunk_text": chunk_text.lower(),
                    "metadata": metadata
                }
                chunks.append(record)
        return chunks
    
    chunks = []
    
    for x in DATA.get("data", []):
        chunk = build_chunks_from_data(x)
        chunks+=chunk

    if not chunks:
        print("Empty Dataset. No chunks are created")
        return
        
    print("Prepared chunks:", len(chunks))

    index_name = "alphawell"

    # namespace: User_IDXXXYYYY-MM-DD_HH:MM
    now = datetime.now()
    namespace = f"{user_id}XXX{now.strftime('%Y-%m-%d_%H-%M-%S')}"

    try:
        pc = Pinecone(api_key=PINECONE_API_KEY)
        # pc = Pinecone()
        print("Successful init")
    except e:
        print("error initialiing PC", e)
    try:
        existing = pc.list_indexes().names()
    except Exception:
        existing = pc.list_indexes()

    if index_name not in existing:
        try:
            if PINECONE_REGION:
                pc.create_index(
                    name=index_name,
                    dimension=dimension,
                    metric="cosine",
                    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
                )
            else:
                pc.create_index(name=index_name, dimension=dimension, metric="cosine")
            print("Index created:", index_name)
        except Exception as e:
            # fallback: retry without spec (project default region)
            print("Create-with-spec failed, retrying without spec:", str(e))
            pc.create_index(name=index_name, dimension=dimension, metric="cosine",spec=ServerlessSpec(cloud="aws", region="us-east-1"))
            print("Index created (default region):", index_name)
    else:
        print("Index exists:", index_name)

    index = pc.Index(index_name)
        
    def make_id_from_text(text: str, prefix: str = None):
        h = hashlib.sha1(text.encode("utf-8")).hexdigest()[:20]
        return f"{prefix + '_' if prefix else ''}{h}"
    
    BATCH = 20
    total = 0
    for i in range(0, len(chunks), BATCH):
        batch = chunks[i:i+BATCH]
        texts = [f"medicine: {c['name']} - topic: {c['topic']} - context: {c['chunk_text']}" for c in batch]
        embs = embeddings.embed_documents(texts)  # list[list[float]]
        vectors = []
        for rec, emb, text in zip(batch, embs, texts):
            # deterministic id
            vec_id = make_id_from_text(rec["chunk_text"], prefix=user_id)
            meta = rec["metadata"].copy()
            # store original chunk text in metadata too
            meta["chunk_text"] = text
            meta["name"] = rec.get("name")
            meta["topic"] = rec.get("topic")
            vectors.append((vec_id, emb, meta))
            total += 1
        index.upsert(vectors=vectors, namespace=namespace)
        print(f"Upserted batch {(i//BATCH)+1}, items={len(vectors)}")
    
    print(f"Total upserted: {total}")
    print("Index_name:", index_name)
    print("Namespace:", namespace)
    
    # returnable result
    # result = {"namespace": namespace}

    return namespace

# print(push_into_db(DATA, "revanth"))

In [13]:
from fastapi import FastAPI, UploadFile, File, Form
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
# import pinecone
# from langchain.vectorstores import Pinecone

app = FastAPI()

origins=["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)

# Preprocesses the PDF and Creates index in Vector DB
@app.post("/upload")
async def upload_multiple(files: List[UploadFile] = File(...), user_id: str = Form(...)):
    print("User_ID: ", user_id)
    processed = []
    for f in files:
        pdf_bytes = await f.read()
        result = preprocess_text_pdf(pdf_bytes)
        pages_info = result.get("response", [])
        processed.append({"filename": f.filename, "page_info": pages_info})
    text_json = {"response": processed}

    # text to JSON
    json_struct = process_text_json_bulk(text_json, process_all=True)

    # Pushing into Pinecone
    namespace = push_into_db(json_struct, user_id)

    # namespace = "revanthXXX2025-08-25_01-42-39"

    return {
        "namespace": namespace
    }

class RetrieverRequest(BaseModel):
    namespace: str

# @app.post("/setRetriever")
# async def set_retriever(req: RetrieverRequest):
#     docsearch = PineconeVectorStore.from_existing_index(
#         index_name='alphawell',
#         embedding=embeddings,
#         namespace=req.namespace
#     )

#     retriever = docsearch.as_retriever(
#         search_type="similarity",
#         search_kwargs={"k": 10}
#     )

#     global current_retriever
#     current_retriever = retriever

#     return {"message": f"Retriever set for the namespace '{req.namespace}'"}

# class QueryRequest(BaseModel):
#     request: str

# # Agentic AI
# @app.post("/alpha_bot29")
# async def respond(data: QueryRequest):
#     answer = askDeepSeek(query = data.request)
#     return {"answer": answer}

In [14]:
import os
from kaggle_secrets import UserSecretsClient

ngrok_label = "NGROK_AUTH_TOKEN"

ngrok_token = UserSecretsClient().get_secret(ngrok_label)

!ngrok authtoken $ngrok_token

ngrok_tunnel = ngrok.connect(8000)
print("Public URL", ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


INFO:     Started server process [294]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL https://d06023c381d6.ngrok-free.app


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [294]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1921, in _run_once
    handle = self._ready.